In [1]:
import tensorflow as tf
import numpy as np
import struct
from array import array

from flatbuffers.packer import float32
from tensorflow.python.data.experimental.ops.distribute import batch_sizes_for_worker

# my project
from module.conf import PROJECT_DIR
# matplotlib.use("QTAgg")
%matplotlib inline

In [4]:
# tf.config.list_physical_devices('GPU')
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [18]:
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'), 'GPU')

In [19]:
tf.test.gpu_device_name()

''

In [2]:
x = tf.Variable(3.0)

In [3]:
with tf.GradientTape() as tape:
    y = x ** 2
    pass
grad = tape.gradient(y, x)

In [4]:
type(grad.numpy())

numpy.float32

In [5]:
del tape

In [ ]:
# mnist_path = "/data/sample/mnist"
# training_images_filepath = "".join([PROJECT_DIR, mnist_path, "/train-images.idx3-ubyte"])
# training_labels_filepath = "".join([PROJECT_DIR, mnist_path, "/train-labels.idx1-ubyte"])
# test_images_filepath = "".join([PROJECT_DIR, mnist_path, "/t10k-images.idx3-ubyte"])
# test_labels_filepath = "".join([PROJECT_DIR, mnist_path, "/t10k-labels.idx1-ubyte"])

# def read_images_labels(images_filepath, labels_filepath) -> tuple:
#     labels = []
#     with open(labels_filepath, 'rb') as file:
#         magic, size = struct.unpack(">II", file.read(8))
#         if magic != 2049:
#             raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
#         # labels = array("B", file.read())
#         labels = array("B", file.read())

#     with open(images_filepath, 'rb') as file:
#         magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
#         if magic != 2051:
#             raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
#         image_data = array("B", file.read())

#     images = []
#     # for i in range(size):
#     #     images.append([0] * rows * cols)
#     for i in range(size):
#         img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
#         img = img.reshape(28, 28)
#         # images[i][:] = img
#         images.append(img)

#     return images, labels

# def load_data() -> tuple:
#     x_train, y_train = read_images_labels(training_images_filepath, training_labels_filepath)
#     x_test, y_test = read_images_labels(test_images_filepath, test_labels_filepath)
#     return (x_train, y_train),(x_test, y_test)

# (X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
# X_train = np.asarray(X_train-127.5) / 127.5
# y_train = np.asarray(y_train)
# X_test  = np.asarray(X_test-127.5) / 127.5
# y_test  = np.asarray(y_test)

In [12]:
class Generator(tf.keras.Model):
    def __init__(self, noise_dim=100):
        super(Generator, self).__init__()
        self.model = tf.keras.Sequential(layers= [
            tf.keras.layers.InputLayer(input_shape=(noise_dim,)),
            tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(units=28 * 28, activation=tf.nn.tanh),
            tf.keras.layers.Reshape(target_shape=(28, 28)),
        ])
        return

    def call(self, inputs, training=False):
        return self.model(inputs, training=training)

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = tf.keras.Sequential(layers=[
            tf.keras.layers.InputLayer(input_shape=(28, 28)),
            tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid),
        ])

    def call(self, inputs, training=False):
        return self.model(inputs, training=training)


In [13]:
class GAN(tf.keras.Model):
    def __init__(self, noise_dim=100):
        super(GAN, self).__init__()
        self.noise_dim = noise_dim
        self.generator = Generator()
        self.discriminator = Discriminator()

    def compile(self, gen_optimizer, disc_optimizer, loss_fn):
        super(GAN, self).compile()
        self.gen_optimizer = gen_optimizer
        self.disc_optimizer = disc_optimizer
        self.loss_fn = loss_fn
        pass

    def train_step(self, batch):
        batch_sizes = tf.shape(batch)[0]
        # random noise
        noise = tf.random.normal(shape=(batch_sizes, self.noise_dim))
        # gen fake img by Generator
        fake_images = self.generator(noise, training=True)
        # 1 - real image, 0 - fake image
        real_labels = tf.ones((batch_sizes, 1))
        fake_labels = tf.zeros((batch_sizes, 1))

        # train Discriminator
        with tf.GradientTape() as disc_tape:
            real_preds = self.discriminator(batch, training=True)
            fake_preds = self.discriminator(fake_images, training=True)

            real_loss = self.loss_fn(real_labels, real_preds)
            fake_loss = self.loss_fn(fake_labels, fake_preds)

            disc_loss = (real_loss + fake_loss) / 2
            pass
        grad = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        self.disc_optimizer.apply_gradients(zip(grad, self.discriminator.trainable_variables))

        # train Generator
        with tf.GradientTape() as gen_tape:
            fake_images = self.generator(noise, training=True)
            fake_preds = self.discriminator(fake_images, training=True)
            gen_loss = self.loss_fn(real_labels, fake_preds)
            pass
        grad = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        self.gen_optimizer.apply_gradients(zip(grad, self.generator.trainable_variables))
        return {"gen_loss": gen_loss, "disc_loss": disc_loss}

In [ ]:
# tf.reshape(X_train, [-1, 784]).numpy().shape

NameError: name 'X_train' is not defined

In [15]:
# dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(60000).batch(128)
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype("float32") - 127.5) / 127.5  # Standartize to [-1, 1]
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60000).batch(128)

11490434/11490434 [==============================] - 2s 0us/step


In [16]:
gan = GAN(noise_dim=100)

gan.compile(
    gen_optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    disc_optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy()
)

gan.fit(dataset, epochs=10000)

Epoch 1/10000
 72/469 [===>..........................] - ETA: 1:40 - gen_loss: 3.1123 - disc_loss: 0.2632

KeyboardInterrupt: 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
import matplotlib.pyplot as plt

def generate_and_show_images(generator, num_examples=10):
    noise = tf.random.normal(shape=(num_examples, 100))
    images = generator(noise, training=False)
    images = (images + 1) / 2.0  # standardize to [0, 1]

    fig, axes = plt.subplots(1, num_examples, figsize=(10, 2))
    for i in range(num_examples):
        axes[i].imshow(images[i], cmap="gray")
        axes[i].axis("off")
    plt.show()

generate_and_show_images(gan.generator)